In [1]:
from huggingface_hub import notebook_login

In [2]:
notebook_login()

In [3]:
from datasets import load_dataset

In [7]:
eli5 = load_dataset("eli5_category", split="train[:5000]")

Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

In [8]:
eli5 = eli5.train_test_split(test_size=0.2)

In [9]:
eli5["train"][0]

{'q_id': '74ohw6',
 'title': 'Asvab scores and what they mean. How do they affect desirability for recruitment?',
 'selftext': '',
 'category': 'Other',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dnzwtu5'],
  'text': ['It\'s pretty simple. They score you on your assorted abilities and aptitude toward certain tasks that you would be doing in your military career. High scores = high aptitude. Recruiters have quotas to meet so if they find someone with a high asvab score, they\'re going to press that person to join up because they\'re much less likely to be disqualified down the road. I\'ve been told that the recruiters get "points" based on where their recruitee goes in the career. Like they\'d get 1 point for an 11B ground pounder and 10 points for nuclear power candidate. Anecdote: I scored the highest score out of my entire high school class when I took the asvab senior year. I had every branch except the Marines practically pounding down my door. Army wanted me in the 

In [10]:
from transformers import AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [11]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/home/geshi/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
eli5 = eli5.flatten()

In [13]:
eli5["train"][0]

{'q_id': '74ohw6',
 'title': 'Asvab scores and what they mean. How do they affect desirability for recruitment?',
 'selftext': '',
 'category': 'Other',
 'subreddit': 'explainlikeimfive',
 'answers.a_id': ['dnzwtu5'],
 'answers.text': ['It\'s pretty simple. They score you on your assorted abilities and aptitude toward certain tasks that you would be doing in your military career. High scores = high aptitude. Recruiters have quotas to meet so if they find someone with a high asvab score, they\'re going to press that person to join up because they\'re much less likely to be disqualified down the road. I\'ve been told that the recruiters get "points" based on where their recruitee goes in the career. Like they\'d get 1 point for an 11B ground pounder and 10 points for nuclear power candidate. Anecdote: I scored the highest score out of my entire high school class when I took the asvab senior year. I had every branch except the Marines practically pounding down my door. Army wanted me in t

In [14]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [15]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1372 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1039 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2063 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1833 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1308 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1164 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2041 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2145 > 1024). Running this sequence through the model will result in indexing errors


In [16]:
block_size = 128

In [17]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [18]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
lm_dataset['train'][0]

{'input_ids': [1026,
  338,
  2495,
  2829,
  13,
  1119,
  4776,
  345,
  319,
  534,
  46603,
  7883,
  290,
  15409,
  3984,
  3812,
  1728,
  8861,
  326,
  345,
  561,
  307,
  1804,
  287,
  534,
  2422,
  3451,
  13,
  3334,
  8198,
  796,
  1029,
  15409,
  3984,
  13,
  3311,
  4872,
  364,
  423,
  38736,
  284,
  1826,
  523,
  611,
  484,
  1064,
  2130,
  351,
  257,
  1029,
  355,
  85,
  397,
  4776,
  11,
  484,
  821,
  1016,
  284,
  1803,
  326,
  1048,
  284,
  4654,
  510,
  780,
  484,
  821,
  881,
  1342,
  1884,
  284,
  307,
  40650,
  866,
  262,
  2975,
  13,
  314,
  1053,
  587,
  1297,
  326,
  262,
  10960,
  364,
  651,
  366,
  13033,
  1,
  1912,
  319,
  810,
  511,
  8921,
  578,
  68,
  2925,
  287,
  262,
  3451,
  13,
  4525,
  484,
  1549,
  651,
  352,
  966,
  329,
  281,
  1367,
  33,
  2323,
  14896,
  263,
  290,
  838,
  2173,
  329,
  4523,
  1176,
  4540,
  13,
  317,
  710,
  10210,
  1258,
  25],
 'attention_mask': [1,
  1,
  1,
  1,
 

In [20]:
from transformers import DataCollatorForLanguageModeling

In [21]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Train

In [22]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

In [23]:
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [24]:
training_args = TrainingArguments(
    output_dir="distilgpt2_eli5_clm-model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [26]:
trainer.train()

[2024-08-30 14:18:28,184] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/geshi/.netrc


/home/geshi/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,3.972900,3.827209
2,3.907900,3.817635
3,3.869200,3.813972


/home/geshi/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/geshi/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/geshi/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/geshi/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tens

TrainOutput(global_step=1995, training_loss=3.8992319054472118, metrics={'train_runtime': 401.3359, 'train_samples_per_second': 79.43, 'train_steps_per_second': 4.971, 'total_flos': 1041202226921472.0, 'train_loss': 3.8992319054472118, 'epoch': 3.0})

In [27]:
import math

In [28]:
eval_results = trainer.evaluate()

In [29]:
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 45.33


In [30]:
trainer.push_to_hub()

events.out.tfevents.1725052708.cajal.1790265.0:   0%|          | 0.00/7.09k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1725053127.cajal.1790265.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/geshijoker/distilgpt2_eli5_clm-model/commit/9097a497a0058c4a5643845c7d2f44c1f46df341', commit_message='End of training', commit_description='', oid='9097a497a0058c4a5643845c7d2f44c1f46df341', pr_url=None, pr_revision=None, pr_num=None)

# Inference

In [31]:
prompt = "Somatic hypermutation allows the immune system to"

In [32]:
from transformers import pipeline

In [33]:
pipe = pipeline("text-generation", model="geshijoker/distilgpt2_eli5_clm-model")

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
pipe(prompt)

# Inference with pytorch

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("geshijoker/distilgpt2_eli5_clm-model")
inputs = tokenizer(prompt, return_tensors="pt").input_ids

In [ ]:
model = AutoModelForCausalLM.from_pretrained("geshijoker/distilgpt2_eli5_clm-model")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=True, top_k=50, top_p=0.95)

In [ ]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)